# script to prepare the files for chromatin accessibility QTL analysis

genotypes are only available through protected access

QTL was generated using QTLtools

In [1]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
os.chdir("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis")
base_dir = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis"

## prepare the normalized counts for caQTL

In [2]:
ATAC_normalised_counts = pd.read_csv(f"{base_dir}/ATAC_seq_analysis/ATAC_DESeq2_quantile_normalized_counts.csv", index_col = 0)
metadata_ATAC = pd.read_csv(f"{base_dir}/metadata/cleaned_ATAC_metadata.csv", index_col=0)
gtf_annotation_df = pd.read_pickle(f"{base_dir}/gencode_gtf.pickle")
gtf_transcripts = gtf_annotation_df[(gtf_annotation_df["feature"] == "transcript") & (gtf_annotation_df["transcript_type"] == "protein_coding")].dropna(axis=1, how='all')
gtf_transcripts["gene_id"] = gtf_transcripts["gene_id"].str.split(".").str[0]
gtf_transcripts["transcript_id"] = gtf_transcripts["transcript_id"].str.split(".").str[0]
gtf_transcripts["TSS_start"] = gtf_transcripts.apply(lambda x: int(x["start"]) if x["strand"] == "+" else int(x["end"]) ,axis = 1)
gtf_genes = gtf_annotation_df[(gtf_annotation_df["feature"] == "gene") & (gtf_annotation_df["gene_type"] == "protein_coding")].dropna(axis=1, how='all')
gtf_genes["gene_id"] = gtf_genes["gene_id"].str.split(".").str[0]
gtf_genes["TSS_start"] = gtf_genes.apply(lambda x: int(x["start"]) if x["strand"] == "+" else int(x["end"]) ,axis = 1)
gtf_genes["length"] = gtf_genes["end"].astype(int) - gtf_genes["start"].astype(int)

In [3]:
ATAC_normalised_counts["gene_id"] = ATAC_normalised_counts.index
ATAC_normalised_counts["length"] = ATAC_normalised_counts["END"] - ATAC_normalised_counts["START"] 
ATAC_normalised_counts["strand"] = "+"

In [4]:
# we don't have genotypes for NRHV319 and NRHV005
samples = metadata_ATAC[~metadata_ATAC["patient"].isin(["NRHV319", "NRHV005"])]["patient"].to_list()

## CD8 counts

In [5]:
wanted_samples = metadata_ATAC[(metadata_ATAC["cell_type"] == "CD8") & metadata_ATAC["patient"].isin(samples)]["id"].to_list()

In [6]:
counts_CD8 = ATAC_normalised_counts[["CHR", "START", "END", "gene_id", "length", "strand"] + wanted_samples].copy()

In [7]:
df_mapper = metadata_ATAC[(metadata_ATAC["cell_type"] == "CD8") & metadata_ATAC["patient"].isin(samples)][["id", "patient"]]

In [8]:
counts_CD8[wanted_samples] = np.log2(counts_CD8[wanted_samples] + 1)

In [9]:
counts_CD8 = counts_CD8.rename(columns = dict(zip(df_mapper['id'], df_mapper['patient'])))
counts_CD8 = counts_CD8.rename(columns = {"CHR":"#chr", "START":"start", "END":"end","gene_id":"gene"})

In [10]:
counts_CD8 = counts_CD8[counts_CD8["#chr"].isin(["chr" + str(i) for i in range(1, 23)])]

In [11]:
counts_CD8.sort_values(["#chr","start"]).to_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/ATAC_CD8_test.bed", sep = "\t", index = False)
!bgzip -f /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/ATAC_CD8_test.bed
!tabix -f -p bed /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/ATAC_CD8_test.bed.gz

## CD4 counts

In [12]:
# we don't have genotypes for NRHV319 and NRHV005
samples = metadata_ATAC[~metadata_ATAC["patient"].isin(["NRHV319", "NRHV005"])]["patient"].to_list()

In [13]:
# PSA4945 and NRHV321 were also identified as outliers
wanted_samples = metadata_ATAC[(metadata_ATAC["cell_type"] == "CD4") & metadata_ATAC["patient"].isin(samples) & (~metadata_ATAC["patient"].isin(["PSA4945", "NRHV321"]))]["id"].to_list()

In [14]:
counts_CD4 = ATAC_normalised_counts[["CHR", "START", "END", "gene_id", "length", "strand"] + wanted_samples].copy()

In [15]:
df_mapper = metadata_ATAC[(metadata_ATAC["cell_type"] == "CD4") & metadata_ATAC["patient"].isin(samples)][["id", "patient"]]

In [16]:
counts_CD4[wanted_samples] = np.log2(counts_CD4[wanted_samples] + 1)

In [17]:
counts_CD4 = counts_CD4.rename(columns = dict(zip(df_mapper['id'], df_mapper['patient'])))
counts_CD4 = counts_CD4.rename(columns = {"CHR":"#chr", "START":"start", "END":"end","gene_id":"gene"})

In [18]:
counts_CD4 = counts_CD4[counts_CD4["#chr"].isin(["chr" + str(i) for i in range(1, 23)])]

In [19]:
counts_CD4.sort_values(["#chr","start"]).to_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/ATAC_CD4_test.bed", sep = "\t", index = False)
!bgzip -f /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/ATAC_CD4_test.bed
!tabix -f -p bed /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/ATAC_CD4_test.bed.gz

QTLtools pca --bed ATAC_CD8_test.bed.gz --scale --center --out ATAC_CD8_test
QTLtools pca --bed ATAC_CD4_test.bed.gz --scale --center --out ATAC_CD4_test

## running a permutation test to identify the best number of PCA component as covariates identified the following:
for CD8: 0 genotype covariates and 10 phenotype covariates
for CD4: 0 genotype covariates and 5 phenotype covariates

In [ ]:
# see bash scripts for running of QTL tools

In [20]:
# merge the final results
!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/output_final/nominal_CD4_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/output_final/ATAC_nominal_CD4_merged.txt
!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/output_final/nominal_CD8_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/output_final/ATAC_nominal_CD8_merged.txt

!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/output_final/permuted_CD4_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/output_final/ATAC_permuted_CD4_merged.txt
!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/output_final/permuted_CD8_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/output_final/ATAC_permuted_CD8_merged.txt

In [2]:
from statsmodels.stats import multitest

## Add FDR correction to permuted p-values

In [3]:
ATAC_permuted_CD8 = pd.read_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/output_final/ATAC_permuted_CD8_merged.txt", sep = " ")
len(ATAC_permuted_CD8[ATAC_permuted_CD8["adj_beta_pval"] < 0.1])

21033

In [4]:
ATAC_permuted_CD8["FDR"] = multitest.fdrcorrection(ATAC_permuted_CD8["adj_beta_pval"], alpha = 0.1)[1]

In [5]:
len(ATAC_permuted_CD8[ATAC_permuted_CD8["FDR"] < 0.1])

7861

In [8]:
ATAC_permuted_CD8.to_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/output_final/ATAC_permuted_CD8_FDR.txt", sep = " ", index = False)

In [9]:
ATAC_permuted_CD4 = pd.read_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/output_final/ATAC_permuted_CD4_merged.txt", sep = " ")
len(ATAC_permuted_CD4[ATAC_permuted_CD4["adj_beta_pval"] < 0.1])

19342

In [10]:
ATAC_permuted_CD4["FDR"] = multitest.fdrcorrection(ATAC_permuted_CD4["adj_beta_pval"], alpha = 0.1)[1]

In [11]:
len(ATAC_permuted_CD4[ATAC_permuted_CD4["FDR"] < 0.1])

6082

In [ ]:
ATAC_permuted_CD4.to_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/ATAC/output_final/ATAC_permuted_CD4_FDR.txt", sep = " ", index = False)